In [4]:
import pandas as pd 
import pickle
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge 
from sklearn.metrics import mean_squared_error
import mlflow
from pathlib import Path

In [2]:
# specify mlflow 
# mlflow.set_tracking_uri("sqlite:///mlflow.db")
# mlflow.set_experiment("nyc-taxi-experiment")

In [3]:
# function to read parquet datasets
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    # preprocess 
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime 
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
# pull training and validation sets
data_path = Path('../data/')
df_train = read_dataframe(filename = data_path/'green_tripdata_2022-01.parquet')
df_val = read_dataframe(filename = data_path/'green_tripdata_2022-02.parquet')
len(df_train), len(df_val)

(59603, 66097)